In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': '188d1eac-0d4f-4c59-b7f7-d247d4854dc0',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 17 Oct 2024 10:04:51 GMT',
   'content-type': 'application/json',
   'content-length': '562',
   'connection': 'keep-alive',
   'x-amzn-requestid': '188d1eac-0d4f-4c59-b7f7-d247d4854dc0',
   'x-amz-apigw-id': 'fygimHamvHcEe6g=',
   'x-amzn-trace-id': 'Root=1-6710e143-3a0eb4b42fe2aff372d3d991'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:767397798699:agent/KYUXQQ71ID',
  'agentId': 'KYUXQQ71ID',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::767397798699:role/c135316a3428926l7994907t1w76739779-BedrockAgentRole-0pZJBLdQYFFR',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 10, 17, 10, 4, 51, 683878, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': '33243896-4a1a-4339-976f-f87208ac644e',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 17 Oct 2024 10:04:56 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '33243896-4a1a-4339-976f-f87208ac644e',
   'x-amz-apigw-id': 'fygjTE4bPHcEm9g=',
   'x-amzn-trace-id': 'Root=1-6710e148-5aeb42eb10fc10674f42b015'},
  'RetryAttempts': 0},
 'agentId': 'KYUXQQ71ID',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 10, 17, 10, 4, 56, 192958, tzinfo=tzlocal())}

In [10]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [12]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [13]:
import uuid

In [14]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [15]:
event_stream = invoke_agent_response["completion"]

In [16]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'GXIJJRCHUB', 'agentId': 'KYUXQQ71ID', 'agentVersion': '1', 'sessionId': 'd455badc-f2e0-4e20-8464-d59e09cbd108', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":"You are an advanced AI agent acting as a front line customer support agent.You have been provided with a set of functions to answer the user\'s question.You must call the functions in the format below:<function_calls>  <invoke>    <tool_name>$TOOL_NAME</tool_name>    <parameters>      <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>      ...    </parameters>  </invoke></function_calls>Here are the functions available:<functions>  </functions>You will ALWAYS follow the below guidelines when you are answering a question:<guidelines>- Think through the user\'s question, extract all data from the question and the previous

{'trace': {'agentAliasId': 'GXIJJRCHUB', 'agentId': 'KYUXQQ71ID', 'agentVersion': '1', 'sessionId': 'd455badc-f2e0-4e20-8464-d59e09cbd108', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 883, 'outputTokens': 89}}, 'rawResponse': {'content': '<thinking>I apologize, but it seems there was an issue with the format of my previous function call. Let me try that again properly.</thinking>\n\n<function_calls>\n  <invoke>\n    <tool_name>GetOrderDetails</tool_name>\n    <parameters>\n      <order_id>123456</order_id>\n    </parameters>'}, 'traceId': '19564c45-744f-481a-bbb1-ff5fd6948bc3-2'}}}}}
{'trace': {'agentAliasId': 'GXIJJRCHUB', 'agentId': 'KYUXQQ71ID', 'agentVersion': '1', 'sessionId': 'd455badc-f2e0-4e20-8464-d59e09cbd108', 'trace': {'orchestrationTrace': {'rationale': {'text': 'I apologize, but it seems there was an issue with the format of my previous function call. Let me try that again properly.', 'traceId': '19564c45-744f-481a-bbb1

{'trace': {'agentAliasId': 'GXIJJRCHUB', 'agentId': 'KYUXQQ71ID', 'agentVersion': '1', 'sessionId': 'd455badc-f2e0-4e20-8464-d59e09cbd108', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 1404, 'outputTokens': 90}}, 'rawResponse': {'content': 'I apologize, but it seems there is still an issue with the function call format that I am using. Let me try to clarify the proper format one more time:\n\n<function_calls>\n  <invoke>\n    <tool_name>GetOrderDetails</tool_name>\n    <parameters>\n      <order_id>123456</order_id>\n    </parameters>'}, 'traceId': '19564c45-744f-481a-bbb1-ff5fd6948bc3-4'}}}}}
{'trace': {'agentAliasId': 'GXIJJRCHUB', 'agentId': 'KYUXQQ71ID', 'agentVersion': '1', 'sessionId': 'd455badc-f2e0-4e20-8464-d59e09cbd108', 'trace': {'orchestrationTrace': {'rationale': {'text': 'I apologize, but it seems there is still an issue with the function call format that I am using. Let me try to clarify the proper format one more time:

In [17]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [18]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's see how I can assist you with returning the mug you
  purchased. I will need to gather some more information to process
  the return properly.

Agent's thought process:
  To process your return, I will need some more details about your
  order. Could you please provide the order ID or the date you made
  the purchase?

Agent's thought process:
  I apologize, but I'm unable to retrieve your order details without
  more information. Could you please provide the order ID or the date
  you made the purchase? That will help me look up the details and
  process your return request.

Observation:
  Type: FINISH

Final response:
   I apologize, but I'm unable to retrieve your order details without
  more information. Could you please provide the order ID or the date
  you made the purchase? That will help me look up the details and
  process your return requ